In [1]:
# pip install websockets pylsl

In [ ]:
import asyncio
import websockets
import json
from pylsl import StreamInfo, StreamOutlet

# Create an LSL stream
info = StreamInfo('WebSocketLSL', 'Markers', 1, 0, 'string', 'myuid12345')
outlet = StreamOutlet(info)

# Mapping specific to LSL Values - Need to map values to events to assign LSL number
disaster_mapping = {
    ("Earthquake-Induced Landslide", "Rumoi"): 1,
    ("Earthquake-Induced Landslide", "Abashiri"): 2,
    ("Earthquake-Induced Landslide", "Hiyama"): 3,
    ("Collapsed Highway Tunnel", "Sorachi"): 4,
    ("Collapsed Highway Tunnel", "Abashiri"): 5,
    ("Power Grid Failure", "Ishikari"): 6,
    ("Power Grid Failure", "Rumoi"): 7,
    ("Flooded Residential Zone", "Tokachi"): 8,
    ("Limited Medical Supplies", "Iburi"): 9,
    ("Blocked Rural Roads", "Shiribeshi"): 10,
    ("Lack of Clean Drinking Water", "Kamikawa"): 11,
    ("Hospital Collapse", "Abashiri"): 12,
    ("Dam Overflow", "Oshima"): 13,
    ("Community Shelters Overcrowded", "Nemuro"): 14,
    ("Disrupted Supply Chains", "Hidaka"): 15,
    ("Malfunctioning Communication Towers", "Tokachi"): 16,
    ("Malfunctioning Communication Towers", "Sorachi"): 17,
    ("Tsunami Aftermath", "Tokachi"): 18,
    ("Damaged Water Treatment Facility", "Rumoi"): 19,
    ("Wildfire Approaching Town", "Kushiro"): 20,
    ("Overcrowded Evacuation Shelters", "Sorachi"): 21,
    ("Bridge Collapse", "Hidaka"): 22,
    ("Broken Gas Pipeline", "Soya"): 23,
    ("Food Shortages in Remote Areas", "Hiyama"): 24,
    ("Volunteer Fatigue", "Tokachi"): 25,
    ("Tsunami Aftermath", "Nemuro"): 26,
}

simulation_event_counter = 0

# Addresses Goal 1: Start of a disaster (resolution_status = in_progress)
# Addresses Goal 2: End of a disaster (resolution_status = resolved)
def process_disaster_event(data):
    
    # Pull additional data from JSON
    event_data = data.get("data", {})
    
    name = event_data.get("name")
    region_id = event_data.get("region_id")
    status = event_data.get("resolution_status").lower()
    
    # Identify the mapped event from JSON data
    key = (name, region_id)
    
    # What if there is no corresponding mapped event
    if key not in disaster_mapping:
        return f"Practice round disaster: {name} in {region_id}"

    # Corresponding number in mapping
    disaster_marker = disaster_mapping[key]
    
    # Disaster Start
    if status.lower() == "not_started":
        return disaster_marker * 2
    # Disaster End
    elif status.lower() == "resolved":
        return disaster_marker * 2 + 1
    # Disaster In Progress
    else:
        return f"Disaster In Progress: {name} in {region_id}"

# Addresses Goal 3: Which resources moved by which participants
def process_deployment_event(data):
    
    # Pull additional data from JSON
    event_data = data.get("data", {})
    resource_id = event_data.get("resource_id")
    
    user_data = data.get("sending_user", {})
    nickname = user_data.get("nickname").lower()
    
    # Is the resource_id a number
    try:
        index = int(resource_id)

        # All 10 resources are assigned from 100 to 190
        if 1 <= index <= 10:
            marker = 90 + index * 10
        
        # If the range is not 01 to 10
        else:
            print(f"Resource ID out of range: {resource_id}")
            return None
    
    # If the resource_id is not a number
    except ValueError:
        print(f"Invalid resource ID: {resource_id}")
        return None
    
    # A way to distinguish between each participant
    # 101 to 194
    if nickname == "ground_1":
        return marker + 1
    elif nickname == "air_1":
        return marker + 2
    elif nickname == "ground_2":
        return marker + 3
    elif nickname == "air_2":
        return marker + 4
    else:
        print(f"Invalid nickname: {nickname}")
        return None
    
# Addresses Goal 4: Start of a round
# Addresses Goal 5: End of a round
def process_simulation_event(data):
    global simulation_event_counter
    
    event_data = data.get("data", {})
    simulation_status = event_data.get("simulation_status").lower()

    # Only "started" and "ended" get markers
    marker = None
    if simulation_status == "started":
        round_number = simulation_event_counter // 4 + 1
        marker = 200 + (round_number - 1)
        simulation_event_counter += 1
    elif simulation_status == "ended":
        round_number = simulation_event_counter // 4 + 1
        marker = 200 + (round_number - 1)
        simulation_event_counter += 1
    elif simulation_status == "ready":
        marker = "SIMULATION READY"

    return marker
    

# Configure LSL Value to send
def get_lsl_marker(data):
    
    event_type = data.get("event_type", "").strip()
    marker = None
    
    # Disaster Event
    if event_type == "DisasterEvent":
        marker = process_disaster_event(data)
    # Deployment of Resources Event
    elif event_type == "DeploymentEvent":
        marker = process_deployment_event(data)
    # Rounds Start and Ends
    elif event_type == "SimulationStatusEvent":
        marker = process_simulation_event(data)
        
    return marker

# WebSocket to receive JSONs
# Accept a variable number of arguments so it works with either 1 or 2 parameters.
async def handle_connection(websocket, *args):

    # If a path is provided, use it; otherwise, default to None
    path = args[0] if args else None
    print(f"Client connected. Path: {path}")

    try:
        async for message in websocket:
            print(f"Received WebSocket message: {message}")

            try:
                data = json.loads(message)

                # Pass the data along to receive LSL value
                marker = get_lsl_marker(data)
                if marker is None:
                    print(f"Unknown event: {marker}")
                    marker = "UNKNOWN_EVENT"
                else:
                    marker = str(marker)

                # Push the event to LSL
                outlet.push_sample([marker])
                print(f"Sent to LSL: {marker}")

                # Send acknowledgment back to the client
                await websocket.send(f"LSL Received: {marker}")

            except json.JSONDecodeError:
                error_msg = "Error: Received non-JSON message"
                print(error_msg)
                await websocket.send(json.dumps({"error": error_msg}))
                
            # Do not want the server disconnecting when there is an error 
            except Exception as e:
                # Log internal errors and notify the sender
                error_msg = f"Internal error: {str(e)}"
                print(error_msg)
                await websocket.send(json.dumps({"error": error_msg}))
               
    except websockets.exceptions.ConnectionClosedOK:
        print("Client disconnected normally.")
        
    except websockets.exceptions.ConnectionClosedError as e:
        print(f"Connection closed with error: {e}")
        
    except Exception as e:
        print(f"Unexpected server error: {e}")

# Run server
async def start_server():
    print("Starting WebSocket Server on ws://localhost:8765")
    
    async with websockets.serve(handle_connection, "localhost", 8765):
        # Keep the server running
        await asyncio.Future()

if __name__ == "__main__":
    await start_server()

Starting WebSocket Server on ws://localhost:8765
Client connected. Path: None
Received WebSocket message: {
    "id": "6d9bf521b5c3405a8649f1c8edc07c3c",
    "sending_user": {
        "id": "2d0bfd0268994737b06102c23a0c9b40",
        "nickname": "Andy",
        "role": "ADMIN"
    },
    "receiving_user": {
        "id": "e62b38f102b4470f9bb573bda93bed21",
        "nickname": "air_1",
        "team": "air",
        "role": "PARTICIPANT",
        "alias": "air_1"
    },
    "event_type": "DisasterEvent",
    "experiment_session_id": "d9d9e7592cb5497a9bb01cbb972564e5",
    "simulation_round_id": "ff8835a2b9544c24849e59ddbbaf6802",
    "data": {
        "description": "A tsunami has devastated coastal areas, leaving many stranded and in need of evacuation.",
        "disaster_resolution": {
            "resources_used": [
                {
                    "available": false,
                    "region_id": "Nemuro",
                    "resource_id": "09",
                    "resour